In [1]:
import pandas as pd
import numpy as np

In [2]:
path='/kaggle/input/pima-indians-diabetes-database/diabetes.csv'
df=pd.read_csv(path)

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [8]:
df.duplicated().sum()

0

In [9]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [10]:
X = df[['Pregnancies', 'Glucose', 'Insulin', 'BMI', 'Age', 'DiabetesPedigreeFunction']]
y = df['Outcome']
print("X (features):")
print(X)

print("\ny (target):")
print(y)

X (features):
     Pregnancies  Glucose  Insulin   BMI  Age  DiabetesPedigreeFunction
0              6      148        0  33.6   50                     0.627
1              1       85        0  26.6   31                     0.351
2              8      183        0  23.3   32                     0.672
3              1       89       94  28.1   21                     0.167
4              0      137      168  43.1   33                     2.288
..           ...      ...      ...   ...  ...                       ...
763           10      101      180  32.9   63                     0.171
764            2      122        0  36.8   27                     0.340
765            5      121      112  26.2   30                     0.245
766            1      126        0  30.1   47                     0.349
767            1       93        0  30.4   23                     0.315

[768 rows x 6 columns]

y (target):
0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766 

In [11]:
from sklearn.preprocessing import StandardScaler

In [12]:
scaler = StandardScaler()

In [13]:
X= scaler.fit_transform(X)

In [14]:
X.shape

(768, 6)

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [16]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [17]:
model= Sequential()
model.add(Dense(32,activation='relu',input_dim=6))
model.add(Dense(1,activation='sigmoid'))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                224       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 257 (1.00 KB)
Trainable params: 257 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [21]:
model.fit(X_train,y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4878 - accuracy: 0.7622 - val_loss: 0.4763 - val_accuracy: 0.7987
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4833 - accuracy: 0.7671 - val_loss: 0.4710 - val_accuracy: 0.8052
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4794 - accuracy: 0.7736 - val_loss: 0.4682 - val_accuracy: 0.8052
Epoch 4/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4760 - accuracy: 0.7704 - val_loss: 0.4663 - val_accuracy: 0.8052
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4730 - accuracy: 0.7671 - val_loss: 0.4645 - val_accuracy: 0.7987
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4705 - accuracy: 0.7687 - val_loss: 0.4608 - val_accuracy: 0.8052
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4686 - accuracy: 0.7638 - val_loss: 0.4599 - val_accuracy: 0.7987
Epoch 8/100
2

# Finding best optimizer using Keras-Tuner

In [22]:
pip install -U keras-tuner

  Obtaining dependency information for keras-tuner from https://files.pythonhosted.org/packages/2b/39/21f819fcda657c37519cf817ca1cd03a8a025262aad360876d2a971d38b3/keras_tuner-1.4.6-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: keras-tuner
    Found existing installation: keras-tuner 1.3.5
    Uninstalling keras-tuner-1.3.5:
      Successfully uninstalled keras-tuner-1.3.5
Note: you may need to restart the kernel to use updated packages.


In [45]:
import keras_tuner as kt

In [51]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(hp):
    model = Sequential()
    model.add(Dense(32, activation='relu', input_dim=6))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [53]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [54]:
tuner.search(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

In [56]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [57]:
model = tuner.get_best_models(num_models=1)[0]

In [58]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                224       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 257 (1.00 KB)
Trainable params: 257 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5099 - accuracy: 0.7704 - val_loss: 0.5069 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4987 - accuracy: 0.7769 - val_loss: 0.4981 - val_accuracy: 0.7727
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4904 - accuracy: 0.7752 - val_loss: 0.4912 - val_accuracy: 0.7727
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4841 - accuracy: 0.7736 - val_loss: 0.4873 - val_accuracy: 0.7662
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4801 - accuracy: 0.7720 - val_loss: 0.4844 - val_accuracy: 0.7662
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4766 - accuracy: 0.7736 - val_loss: 0.4823 - val_accuracy: 0.7662
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4733 - accuracy: 0.7736 - val_loss: 0.4802 - val_accuracy: 0.7662
Epoch 14

# Using Keras-Tuner to find best number of neurons in a particular layer

In [95]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(hp):
    model = Sequential()
    units = hp.Int('random_units', min_value=4, max_value=128)
    model.add(Dense(units=units, activation='relu', input_dim=6))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model




In [99]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=15, directory='mydir',project_name='my_tuner')

Reloading Tuner from mydir/my_tuner/tuner0.json


In [100]:
tuner.search(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

Trial 15 Complete [00h 00m 04s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 06m 39s


In [101]:
tuner.get_best_hyperparameters()[0].values

{'random_units': 128}

In [102]:
model = tuner.get_best_models(num_models=1)[0]

In [103]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.4444 - accuracy: 0.7818 - val_loss: 0.4566 - val_accuracy: 0.8117
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4433 - accuracy: 0.7801 - val_loss: 0.4580 - val_accuracy: 0.8117
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4433 - accuracy: 0.7801 - val_loss: 0.4570 - val_accuracy: 0.8182
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4427 - accuracy: 0.7818 - val_loss: 0.4580 - val_accuracy: 0.8117
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4429 - accuracy: 0.7899 - val_loss: 0.4562 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4416 - accuracy: 0.7818 - val_loss: 0.4566 - val_accuracy: 0.8117
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4414 - accuracy: 0.7834 - val_loss: 0.4573 - val_accuracy: 0.8182
Epoch 14

# Finding appropiate number of layers using Keras-Tunner

In [108]:
def build_model(hp):
    model= Sequential()
    model.add(Dense(80,activation='relu',input_dim=6))
    for i in range (hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(72,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [109]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=15, directory='mydir2',project_name='my_tuned_layers')

In [110]:
tuner.search(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

Trial 10 Complete [00h 00m 06s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 01m 02s


In [111]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4}

In [112]:
model = tuner.get_best_models(num_models=1)[0]

In [113]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 17ms/step - loss: 0.4190 - accuracy: 0.7850 - val_loss: 0.4733 - val_accuracy: 0.8247
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4176 - accuracy: 0.7915 - val_loss: 0.4883 - val_accuracy: 0.8052
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4124 - accuracy: 0.7964 - val_loss: 0.4814 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4114 - accuracy: 0.7850 - val_loss: 0.4722 - val_accuracy: 0.8377
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4023 - accuracy: 0.8046 - val_loss: 0.4859 - val_accuracy: 0.8117
Epoch 12/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3980 - accuracy: 0.7997 - val_loss: 0.4998 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 5ms/step - loss: 0.3908 - accuracy: 0.8094 - val_loss: 0.5114 - val_accuracy: 0.8052
Epoch 14